In [112]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.examples.tutorials.mnist import input_data
from matplotlib import image
from matplotlib import pyplot as plt
import numpy as np
import os

In [4]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## conv

In [18]:
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.3)
    return tf.Variable(initial)

def conv2d(x, W, strides=[1, 1, 1, 1]):
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [19]:
def add_histogram(var, name):
    with tf.name_scope('summaries'):
        tf.summary.histogram(name, var)

In [100]:
def make_sprite(imgs, out_path):
    """
    imgs: 2D (!) numpy array containing MNIST image data (i.e. 784-length vectors).
    out_path: Path to store sprite image at (e.g. path/to/file.png).
    """
    imgs_square = imgs.reshape(-1, 28, 28)
    grid_dim = int(np.ceil(np.sqrt(len(imgs))))
    grid_rows = [np.concatenate(imgs_square[ind:(ind+grid_dim)], axis=1) for ind in range(0, len(imgs), grid_dim)]
    # maybe pad last row
    missing = grid_rows[0].shape[1] - grid_rows[-1].shape[1]
    if missing:
        grid_rows[-1] = np.pad(grid_rows[-1], ((0, 0), (0, missing)), mode="constant")
    grid = np.concatenate(grid_rows, axis=0)
    # note that the images are reversed (black on white background) for readability
    image.imsave(out_path, grid, cmap="Greys")

In [68]:
tf.reset_default_graph() 

# build computation graph
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_img = tf.reshape(x, [-1, 28, 28, 1])

strides = [1, 1, 1, 1]

with tf.name_scope('conv_1') as scope:
    W_conv1 = weight_var([3, 3, 1, 32])
    b_conv1 = bias_var([32])
    h_conv1 = tf.nn.relu(conv2d(x_img, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    add_histogram(W_conv1, 'w_conv_1')
    add_histogram(b_conv1, 'b_conv1')

with tf.name_scope('conv_2') as scope:
    W_conv2 = weight_var([7, 7, 32, 64])
    b_conv2 = bias_var([64])
    
    add_histogram(W_conv2, 'w_conv_2')
    add_histogram(b_conv2, 'b_conv_2')
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

with tf.name_scope('fc_1') as scope:
    W_fc1 = weight_var([7*7*64, 1024])
    b_fc1 = bias_var([1024])
    
    add_histogram(W_fc1, 'w_fc1')
    add_histogram(b_fc1, 'b_fc1')
    
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    add_histogram(h_fc1, 'h_fc1')

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# to be run after training to assign the output of fc_1 to another variable
# for visualization
# embedding_vec = tf.get_variable("embeddings", [500, 1024])
# save_embeddings = tf.assign(embedding_vec, h_fc1_drop)
    
with tf.name_scope('output') as scope:
    W_fc2 = weight_var([1024, 10])
    b_fc2 = bias_var([10])

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
        
embedding_vec = tf.Variable([0.], name='embedding')
save_embeddings = tf.assign(embedding_vec, h_fc1, validate_shape=False)
        
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

In [130]:
summaries_dir = '../nov13/embeddings/1'

In [131]:
%%time

saver = tf.train.Saver()
num_to_viz = 400

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sesh:
    train_writer = tf.summary.FileWriter(summaries_dir + '/train', sesh.graph)
    test_writer = tf.summary.FileWriter(summaries_dir + '/test')
    
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embedding_vec.name
    embedding_conf.metadata_path = 'metadata.tsv'
    embedding_conf.sprite.image_path = 'sprite.jpg'
    embedding_conf.sprite.single_image_dim.extend([28, 28])
    projector.visualize_embeddings(test_writer, config)

    sesh.run(tf.global_variables_initializer())
    batch_size = 50
    
    test_xs, test_ys = mnist.test.images, mnist.test.labels
    
    for i in range(1000):
        batch = mnist.train.next_batch(batch_size)
        if i % 50 == 0: # record test set accuracy
            summary, acc = sesh.run([merged, accuracy], feed_dict={x: test_xs, y_: test_ys, keep_prob: 1.})
            test_writer.add_summary(summary, i)
        else:
            if i % 100 == 99: # record train set accuracy
                sesh.run([save_embeddings], feed_dict={x: test_xs[:num_to_viz, :],
                                                       y_: test_ys[:num_to_viz, :]})
                saver.save(sesh, os.path.join(summaries_dir, "model.ckpt"))
                
                summary, _ = sesh.run([merged, train_step], 
                                      feed_dict={x: batch[0],
                                                 y_: batch[1],
                                                 keep_prob: 0.5},
                                      options=run_options,
                                      run_metadata=run_metadata)

                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:  # Record a summary
                sesh.run([train_step], feed_dict={x: batch[0],
                                                 y_: batch[1],
                                                 keep_prob: 0.5})
        

test_writer.close()
train_writer.close()


Adding run metadata for 99
Adding run metadata for 199
Adding run metadata for 299
Adding run metadata for 399
Adding run metadata for 499
Adding run metadata for 599
Adding run metadata for 699
Adding run metadata for 799
Adding run metadata for 899
Adding run metadata for 999
CPU times: user 31.4 s, sys: 2.14 s, total: 33.5 s
Wall time: 30.8 s


In [132]:
# save labels as metadata
with open(os.path.join(summaries_dir + '/test', 'metadata.tsv'), 'w') as fout:
    for label in list(np.where(test_ys==1)[1][:num_to_viz]):
        fout.write('{}\n'.format(label))         
        
make_sprite(test_xs[:num_to_viz, :], os.path.join(summaries_dir + '/test', 'sprite.jpg'))